In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from seaborn import heatmap
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import svm, tree, neighbors, ensemble, gaussian_process

In [ ]:
nClass = 2
Classes = list()
for n in range(nClass):
    Classes.append(f"Class {n}")
print(f"Class Number : {nClass}\n{Classes}")

In [ ]:
ablations = ['', '-PVI', '-BOX']
fibrosis = ['50PF', '70PF', '71PF']
fibnames = ['moderate fibrosis', 'severe fibrosis - case 1', 'severe fibrosis - case 2']
abnames = ['', ' - PVI', ' - PVI + BOX']
Cases = []
for f in fibrosis:
    for a in ablations:
        Cases.append(f + a)
Names = []
for f in fibnames:
    for a in abnames:
        Names.append(f + a)
IdCase = 3
print(f"      cases : {Cases}")
print(f"      names : {Names}")
print(f"picked case : {Names[IdCase]}")

In [ ]:
N_H = 40
N_L = 100
Path = "../Datasets/"
ChoosePoints = True
Y_L_all = np.load(Path + "LF_train-%s.npz" % Cases[IdCase])["output"]
Y_L = Y_L_all[:N_L]
Y_H_all = np.load(Path + "HF_train-%s.npz" % Cases[IdCase])["output"]
Y_H = Y_H_all[:N_H]
y_true = np.load(Path + "ground_truth-%s.npz" % Cases[IdCase])["output"]
TrainPoints = np.genfromtxt(Path + "train_points.csv").astype(float)
TruePoints = np.genfromtxt(Path + "ground_truth_points.csv").astype(float)
if ChoosePoints:
    X_true = TruePoints[:N_L,:3]
    X_L = TrainPoints[:N_L,:3]
    X_H = TrainPoints[:N_H,:3]
else:
    X_true = TruePoints[:N_L,3]
    X_L = TrainPoints[:N_L,3]
    X_H = TrainPoints[:N_H,3]

In [ ]:
[Y_L_all.shape, Y_H_all.shape]

In [ ]:
[X_true.shape, y_true.shape, X_L.shape, Y_L.shape, X_H.shape, Y_H.shape]

In [ ]:
SecLH = 1
if SecLH == 0:
    xTrain, yTrain = X_L, Y_L
else:
    xTrain, yTrain = X_H, Y_H
xTest = X_true
yTest = y_true
if len(xTrain.shape) == 1:
    xTrain = np.expand_dims(xTrain, axis=-1)
    xTest = np.expand_dims(xTest, axis=-1)
[xTrain.shape, yTrain.shape, xTest.shape, yTest.shape]

In [ ]:
xMin, xMax = xTrain.min(axis=0), xTrain.max(axis=0)
xTrain = (xTrain-xMin) / (xMax-xMin)
xTest = (xTest-xMin) / (xMax-xMin)
[xTrain.min(axis=0), xTrain.max(axis=0), xTest.min(axis=0), xTest.max(axis=0)]

In [ ]:
SecModel = 1
if SecModel == 0:
    print("Gaussian Process Classifier")
    ClassModel = gaussian_process.GaussianProcessClassifier()
elif SecModel == 1:
    print("Decision Tree")
    ClassModel = tree.DecisionTreeClassifier()
elif SecModel == 2:
    print("Random Forest")
    ClassModel = ensemble.RandomForestClassifier(n_estimators=50, warm_start=True)
elif SecModel == 3:
    print("Extra Trees")
    ClassModel = ensemble.ExtraTreesClassifier(n_estimators=50, warm_start=True)
elif SecModel == 4:
    print("KNeighbors")
    ClassModel = neighbors.KNeighborsClassifier(n_neighbors=3)
else:
    print("Support Vector Machine")
    ClassModel = svm.SVC()
ClassModel.fit(xTrain,yTrain)

In [ ]:
nTest = yTest.shape[0]
yTestPred = ClassModel.predict(xTest)
yTestFound = sum(yTest==yTestPred)
yTestPercent = 100*yTestFound/nTest
nTrain = yTrain.shape[0]
yTrainPred = ClassModel.predict(xTrain)
yTrainFound = sum(yTrain==yTrainPred)
yTrainPercent = 100*yTrainFound/nTrain
print(f"Train Accuracy : {yTrainPercent:.6g} % ({yTrainFound} / {nTrain})")
print(f" Test Accuracy : {yTestPercent:.6g} % ({yTestFound} / {nTest})")

In [ ]:
ConMatTrain = confusion_matrix(yTrain, yTrainPred)
ConMatTest = confusion_matrix(yTest, yTestPred)
print("For Train Data", ConMatTrain, sep="\n", end="\n\n")
print("For Test Data", ConMatTest, sep="\n", end="\n\n")

In [ ]:
plt.figure(figsize=(10,4), dpi=100)
plt.subplot(1,2,1)
heatmap(data=ConMatTrain, cmap='Blues', linewidths=1, linecolor='gray',
        fmt='d', annot=True, xticklabels=Classes, yticklabels=Classes)
plt.title("For Train Data", fontsize = 10)
plt.xlabel("Predicted Classes", fontsize = 10)
plt.ylabel("True Classes", fontsize = 10)
plt.subplot(1,2,2)
heatmap(data=ConMatTest, cmap='Blues', linewidths=1, linecolor='gray',
        fmt='d', annot=True, xticklabels=Classes, yticklabels=Classes)
plt.title("For Test Data", fontsize = 10)
plt.xlabel("Predicted Classes", fontsize = 10)
plt.ylabel("True Classes", fontsize = 10)
plt.show()

In [ ]:
print('For Train Data')
print(classification_report(yTrain, yTrainPred, target_names=Classes, digits=4))
print('For Test Data')
print(classification_report(yTest, yTestPred, target_names=Classes, digits=4))